In [1]:
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

def ultimate_get_filename(row, list_filenames, list_paths):
    name = row['Fichier']
    name = name.split('/')[-1]
    name = name.split('.')[0]
    splits = name.split('-')
    current_name = splits[0]
    i = 1
    while current_name not in list_filenames:
        # print(current_name)
        try:
            current_name += '-' + splits[i]
            i += 1
        except:
            row['Dadagp_Path'] = 'error'
            row['File_Name'] = 'error'
            return row
    
    if row['Dadagp_Path'] != 'error':
        index = list_filenames.index(current_name)
        path = list_paths[index]
        row['Dadagp_Path'] = path
        row['File_Name'] = current_name
        return row


In [2]:
df = pd.read_csv('..\\..\\data\\function_guitar-ext-with-predictions.csv')

In [3]:
DADAGPPATH = pathlib.Path(path_to_general_read_folder)

all_paths = list(DADAGPPATH.rglob('**/*.pygp.gp5'))
# retrieve ALL filenames in DadaGP 
all_filenames = [f.name for f in all_paths]
all_filenames_no_ext = [f.name[:-13] for f in all_paths]

index = np.random.randint(0, len(all_paths))

all_paths[index], all_filenames[index], all_filenames_no_ext[index]

(WindowsPath('../../data/DadaGP/DadaGP-v1.1/P/Paradise Lost/Paradise Lost - Walk Away (2).gp3.pygp.gp5'),
 'Paradise Lost - Walk Away (2).gp3.pygp.gp5',
 'Paradise Lost - Walk Away (2)')

In [4]:
tqdm.pandas()
# df.reset_index(drop=True, inplace=True)
# drop Unnamed: 0
df_to_process = df.drop_duplicates(subset=['Fichier']).copy()[['Fichier']]
df_to_process.loc[:, 'File_Name'] = 'Not found'
df_to_process.loc[:, 'Dadagp_Path'] = 'Not found'
df_to_process.head()

,Fichier,File_Name,Dadagp_Path
0,../data/DadaGP8-gpif/ - Isolated.gpif,Not found,Not found
131,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 1....,Not found,Not found
341,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 2....,Not found,Not found
386,../data/DadaGP8-gpif/*.gpif,Not found,Not found
517,../data/DadaGP8-gpif/1 God - Grace.gpif,Not found,Not found


In [5]:
df_to_process = df_to_process.progress_apply(ultimate_get_filename,
                                             axis=1, args=(all_filenames_no_ext, all_paths)) # 2min

100%|██████████| 56574/56574 [01:27<00:00, 648.95it/s]


In [6]:
# Merge back
df = df.merge(df_to_process[['Fichier', 'File_Name', 'Dadagp_Path']], on='Fichier', how='left')

In [7]:
# Count number of errors
display(df[df['Dadagp_Path'] == 'error']['Fichier'].unique())
# Windows doesn't accept files starting with a blank space, we have to rename those

# Remove the duplicates from the dataframe
df = df[df['Dadagp_Path'] != 'error']

array(['../data/DadaGP8-gpif/*.gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (1).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (2).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (3).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (1).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (2).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (3).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (1).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (2).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (3).gpif',
       '../data/DadaGP8-gpif/Irish - Kingfair (bass Solo Version) (1).gpif',
       '../data/DadaGP8-gpif/Joel, Billy - Just the way you are (1).gpif',
       '../data/DadaGP8-gpif/Kaukonen, Jorma - Hesitation Blues (1).gpif',
       '../data/DadaGP8-gpif/Saez - MontCe LO Haut (2) (1).gpif',
       '.

In [8]:
df.reset_index(drop=True, inplace=True)
# drop Unnamed: 0
df = df.drop(columns=['Unnamed: 0'])
display(df.head())
df.shape # 1358 measures removed because of the duplicates (very little)

,Fichier,Track_Name,Partie,Instrument,Mesure,Has_SingleNotes,Has_Chords,Has_2_note_chords,Has_3_note_chords,Has_4_note_chords,...,Letring100pourcents,Nb_VibratoW,Presence_VibratoMG,Nb_Whammy_Bar,Presence_bends,has_triad,has_fifth,rg-estimation,File_Name,Dadagp_Path
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,1,1,0,1,0,...,0,0,0,0,0,1,1,2.570728e-07,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,2,1,1,0,1,0,...,0,0,0,0,0,1,1,4.580104e-07,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,3,1,1,0,1,0,...,0,0,0,0,0,1,1,6.601997e-06,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,4,1,1,0,1,0,...,0,0,0,0,0,1,1,4.111985e-04,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,5,1,1,0,1,0,...,0,0,0,0,0,1,1,4.352387e-05,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...


(4164431, 41)

In [9]:
df = df[['Fichier', 'Dadagp_Path', 'File_Name', 'Track_Name', 'Partie', 'Instrument', 'Mesure', 'rg-estimation']]
df.to_csv('../../data/df_rg_final.csv', index=False)

df.head()

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05
